In [2]:
import pandas as pd
import numpy as np

In [3]:
tx_deaths = pd.read_csv("05_cleaned_data/deaths_tx.csv")
tx_prescriptions = pd.read_csv("05_cleaned_data/prescriptions_tx.csv")

In [4]:
## filter to only texas (only for pre-post)

tx_deaths = tx_deaths[tx_deaths["StateName"] == "Texas"]
tx_prescriptions = tx_prescriptions[tx_prescriptions["BUYER_STATE"] == "TX"]

In [5]:
tx_deaths.head()

,County,County Code,Year,Year Code,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths,State,StateFIPS,CountyName,StateName,CountyFIPS,StateAbbr,STATE_COUNTY,state_abbrev,FIP_unique,Population,county_test
2993,Anderson,48001.0,2004.0,2004.0,Drug poisonings (overdose) Unintentional (X40-...,D1,11.0,TX,48,Anderson,Texas,48001,TX,TX | ANDERSON,TX,4800148,56245.0,Anderson
2994,Anderson,48001.0,2004.0,2004.0,All other non-drug and non-alcohol causes,O9,576.0,TX,48,Anderson,Texas,48001,TX,TX | ANDERSON,TX,4800148,56245.0,Anderson
2995,Anderson,48001.0,2005.0,2005.0,All other non-drug and non-alcohol causes,O9,537.0,TX,48,Anderson,Texas,48001,TX,TX | ANDERSON,TX,4800148,56873.0,Anderson
2996,Anderson,48001.0,2006.0,2006.0,Drug poisonings (overdose) Unintentional (X40-...,D1,10.0,TX,48,Anderson,Texas,48001,TX,TX | ANDERSON,TX,4800148,57386.0,Anderson
2997,Anderson,48001.0,2006.0,2006.0,All other non-drug and non-alcohol causes,O9,580.0,TX,48,Anderson,Texas,48001,TX,TX | ANDERSON,TX,4800148,57386.0,Anderson


In [6]:
# jan 4 2007

target_yr = 2007

In [7]:
# function to select only overdose records
# double check this

def select_overdose(record):

    if record == "All other non-drug and non-alcohol causes":
        return 0

    if record == "All other alcohol-induced causes":
        return 0

    if record == "All other drug-induced causes":
        return 0

    if record == "Alcohol poisonings (overdose) (X45, X65, Y15)":
        return 0

    if record == "Drug poisonings (overdose) Unintentional (X40-X44)":
        return 1

    if record == "Drug poisonings (overdose) Suicide (X60-X64)":
        return 1

    if record == "Drug poisonings (overdose) Undetermined (Y10-Y14)":
        return 1

    else:
        return "error"
    

tx_deaths["overdose"] = tx_deaths["Drug/Alcohol Induced Cause"].apply(lambda x: select_overdose(x))


# filter accordingly
tx_deaths = tx_deaths[tx_deaths["overdose"] != 0]

In [8]:
# normalize deaths over population

tx_deaths["overdose_per_100k"] = tx_deaths["Deaths"] / tx_deaths["Population"] * 100_000

In [9]:
tx_result = tx_deaths.groupby("Year")["overdose_per_100k"].sum()

In [10]:
tx_result

Year
2004.0    210.676289
2005.0    248.943355
2006.0    423.517095
2007.0    280.969382
2008.0    371.347741
2009.0    404.855060
2010.0    426.309075
Name: overdose_per_100k, dtype: float64